In [ ]:
import torch
import torchvision
import torch.nn as nn
import numpy as np
import torchvision.transforms as transforms
import matplotlib.pyplot as plt

# 우리가 사용할 컴퓨터를 check하는 부분, cpu/gpu 지원 받을지...
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
device

In [ ]:
# 하이퍼 파라미터들
input_size = 784  # 28 * 28 * 1 = 784 (픽셀을 1차원으로 늘린다.)
hidden_size = 500 # 은닉 계층의 unit 수(히든 레이어는 하나!)
num_classes = 10 # 카테고리의 개수(deep learning의 output의 카테고리 개수)
num_epochs = 5
batch_size = 100
learning_rate = 0.001

In [ ]:
# 데이터셋 로딩 -- 2번에 걸쳐서 로딩

# 1step(데이터 다운로드)
# train_dataset
train_dataset = torchvision.datasets.MNIST(root="../../data",
                                           train = True,
                                           transform = transforms.ToTensor(),
                                           download = True # 스케일링해서 다운로드하겠다.
                                           )
# test_dataset
test_dataset = torchvision.datasets.MNIST(root="../../data",
                                          train=False,
                                          transform = transforms.ToTensor()
                                          ) # 다운로드는 한번만 하면 된다.

# 2step...DataLoader...BatchSize를 이용함(네트워크로 Load - 모델,머신 안으로 로딩)
train_loader = torch.utils.data.DataLoader(dataset=train_dataset,
                                           batch_size = batch_size,
                                           shuffle = True
                                           ) 

test_loader = torch.utils.data.DataLoader(dataset=test_dataset,
                                           batch_size = batch_size,
                                           shuffle = False # 테스트할 때는 굳이 shuffle 필요 없음
                                           ) 

## Convolution NeuralNet Model 생성

1. VGG모델 표만들기
2. 이 코드가 어떻게 바뀌는지 그림 그리기

In [ ]:
class ConvNet(nn.Module):
    def __init__(self,num_classes=10):
        super(ConvNet, self).__init__()
        self.layer1 = nn.Sequential(
            # 입력 채널수(1->흑백), 출력 채널수, 필터 사이즈, stride, padding
            nn.Conv2d(1,16,kernel_size=5,stride=1,padding=2), # 커널 = 5=> 필터 5x5
            nn.ReLU())
        self.layer2 = nn.MaxPool2d(kernel_size=2, stride=2)
        self.layer3 = nn.Sequential(
            nn.Conv2d(16,32,kernel_size=5,stride=1, padding=2),
            nn.ReLU()
        )
        self.layer4 = nn.MaxPool2d(kernel_size = 2,stride=2)
        self.layer5 = nn.Linear(7*7*32,num_classes) # 인자값 : 입력, 출력
        # CNN에서는 최종 분류할 때 FCN으로 함

    def forward(self,x):
        out = self.layer1(x)
        out = self.layer2(out)
        out = self.layer3(out)
        out = self.layer4(out)
        out = out.reshape(out.size(0),-1)
        out = self.layer5(out)

        return out

In [ ]:
model = ConvNet(num_classes).to(device)
loss_function = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(),lr=learning_rate)

In [ ]:
total_step = len(train_loader)
for epoch in range(num_epochs):
    for i,(images,labels) in enumerate(train_loader):
        images = images.to(device)
        labels = labels.to(device)
        outputs = model(images)
        loss = loss_function(outputs, labels)

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        if (i+1) % 100 == 0:
            print("Epoch[{}/{}], step [{}/{}], Loss:{:.4f}".format(epoch+1,num_epochs,i+1,total_step,loss.item()))

In [ ]:
with torch.no_grad(): # 실제로 학습할 필요가 없을 때 이 구문을 반드시 작성해야 한다.
    correct = 0
    total = 0

    for images, labels in test_loader: # 100개씩 100번 돌아간다.
        images = images.to(device)
        labels = labels.to(device)

        outputs = model(images)
        _,predicted = torch.max(outputs.data,1)

        total += labels.size(0)
        correct += (predicted == labels).sum().item()
        print("Accuracy of the Network on the Test images : {} %".format(100*correct/total))